In [ ]:
import sys
sys.path.append('../src')

from matplotlib import pyplot as plt

from nlkda.utils import MLFlowClient, flatten_dict
from nlkda.data.base import DatasetEnum, get_dataset_size
from nlkda.utils import get_skyline
from nlkda.models.utils import ModelEnum
from nlkda.settings import K_MAX

data_root = Path("/tmp/data/")
exp = data_root / 'experiments'
db = MLFlowClient(root=str(exp), tracking_uri="localhost:5000/")

In [ ]:
def get_additional_size_by_agg_strat(agg_strat, ds):
    additional = K_MAX
    
    if 'point' in agg_strat:
        ds_size = get_dataset_size(DatasetEnum(ds))
        if 'k' in agg_strat:
            additional += 2*ds_size
        else: 
            additional = 2*ds_size
    return additional

In [ ]:
# Plot
markers = ['o', 'P', 'v', 's', 'D']
styles = ['solid', 'dotted', 'dashed', 'dashdot']
for index, ds in enumerate(list(map(lambda c: c.value, DatasetEnum))):
    # plot cop tree
    cop_info = db.unified_get_entries(['params.dataset', 'params.model.model_type'],[True, True],[ds, ModelEnum.COP.value])
    if len(cop_info):
        cop_size = cop_info['metrics.model_size'][0]
        cop_mean = cop_info['metrics.cs_mean'][0]
        cop_median = cop_info['metrics.cs_median'][0]
        plt.plot(cop_size, cop_mean, color="black", marker=markers[index], zorder=100)
    else:
        print('No CoP info in mlflow')
    
    out_dir_string = "params.out_dir"

    # choose one of 'cs_median_mono', 'cs_mean_mono', 'cs_median_mono_point', 'cs_mean_mono_point'
    skyline_col = "metrics.cs_mean_mono_k_point"
    # get skyline
    df = db.unified_get_entries(['params.dataset', 'params.model.model_type'],[True, False],[ds, ModelEnum.COP.value])   
    if len(df):
        df = df.dropna(subset=[skyline_col])
        skyline = get_skyline(df, ["metrics.model_size",skyline_col],[True,True])
        skyline["metrics.model_size"] = skyline["metrics.model_size"] + get_additional_size_by_agg_strat(skyline_col, ds)

        plt.plot(skyline["metrics.model_size"].to_numpy(), skyline[skyline_col].to_numpy(), marker=markers[index], ls=styles[index], label=f'{ds.upper()}', zorder=99)
    else:
        print(f'No models for dataset {ds}')

plt.rcParams['axes.axisbelow'] = True
plt.yscale('log')
plt.xscale('log')
plt.ylabel(f'{skyline_col} [log]')
plt.xlabel('Size [log]')
plt.grid(which='minor', alpha=0.2)
plt.grid(which='major', alpha=1)
plt.legend()
plt.tight_layout()
plt.savefig('model-size-vs-cs.pdf')